In [60]:
# HAMOYE DATA SCIENCE INTERNSHIP QUIZ CODE
# imports necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer

In [61]:
# loads the csv data into a pandas dataframe
raw_data = pd.read_csv("energydata_complete.csv")

In [62]:
#prints out the first 5 rows from the dataset
raw_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [63]:
# gives statistical information about the dataset
raw_data.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [64]:
# gives information about missing rows and column data types
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [65]:
# drops the date and light columns as stated in the question
raw_data.drop(columns=["date", "lights"], inplace=True)

#creates a MinMaxScaler to normalize the dataset
scaler = MinMaxScaler()

# applies normalization to the dataset, sclaes the value between 0 and 1
normalized_data = scaler.fit_transform(raw_data)

# loads the column as a list
columns = raw_data.columns

In [66]:
# loads the normalized data into a new dataframe
raw_data = pd.DataFrame(normalized_data, columns=columns)

#selects the features and labels to use
features = raw_data.drop(columns=["Appliances"])
label = raw_data[['Appliances']]

In [67]:
# splits the data into trainset and test sets
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=42)

In [68]:
# creates a linear regression class
linear_all = LinearRegression()

#trains the data on the linear regression model with the train set
linear_all.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [69]:
# predicts the values on the test set
predict_all = linear_all.predict(x_test)

In [106]:
# prints out the coefficient/weights for each feature
pd.Series(np.array(linear_all.coef_).ravel(), np.array(x_train.columns).ravel()).sort_values()

RH_2          -0.456698
T_out         -0.321860
T2            -0.236178
T9            -0.189941
RH_8          -0.157595
RH_out        -0.077671
RH_7          -0.044614
RH_9          -0.039800
T5            -0.015657
T1            -0.003281
rv1            0.000770
rv2            0.000770
Press_mm_hg    0.006839
T7             0.010319
Visibility     0.012307
RH_5           0.016006
RH_4           0.026386
T4             0.028981
Windspeed      0.029183
RH_6           0.038049
RH_3           0.096048
T8             0.101995
Tdewpoint      0.117758
T6             0.236425
T3             0.290627
RH_1           0.553547
dtype: float64

In [45]:
# calculates the cofficient of determination for the multiple linear regression model
coef_det = r2_score(y_test, predict_all).round(2)

print("The coefficient of determination is {}".format(coef_det))

The coefficient of determination is 0.15


In [49]:
# calculates the mean absolute error for the multiple linear regression model
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predict_all).round(3)
print("The mean absolute error for the multiple Linear Regression Model is {}".format(mae))

The mean absolute error for the multiple Linear Regression Model is 0.05


In [50]:
# calculates the root mean square error for the multiple linear regression model
rmse = np.sqrt(mean_squared_error(y_test, predict_all)).round(3)

print("The root mean square error for the multiple Linear Regression Model is {}".format(rmse))

The root mean square error for the multiple Linear Regression Model is 0.088


In [51]:
# calculates the residual sum of square error for the multiple linear regression model
import numpy as np

rss = np.sum(np.square(y_test - predict_all)).round(2)
print("The residual sum of square error for the multiple linear regression model is {}".format(rss))

The residual sum of square error for the multiple linear regression model is Appliances    45.35
dtype: float64


In [16]:
# trains a simple linear regression model when T2 nnd T6 are features and columns respectively
train_12 = features[["T2"]]
label_12 = features[["T6"]]

#splits into train set and test set
x_train_12, x_test_12, y_train_12, y_test_12 = train_test_split(train_12, label_12, test_size=0.3, random_state=42)

lnr_12 = LinearRegression()
lnr_12.fit(x_train_12, y_train)

In [52]:
# makes prediction on the model
predict_12 = lnr_12.predict(x_test_12)
r2_12 = r2_score(y_test_12, predict_12).round(2)

print("The r2 score for the uni variate model is {}".format(r2_12))

The r2 score for the uni variate model is -3.29


In [53]:
# to apply ridge regression on the dataset
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

rid_pred = ridge_reg.predict(x_test)

# calculates the root mean square error for the ridge model
from sklearn.metrics import mean_squared_error

rmse_r = np.sqrt(mean_squared_error(y_test, rid_pred)).round(3)

print("The root mean square error for the ridge model is {}".format(rmse_r))

The root mean square error for the ridge model is 0.088


In [55]:
# to apply lassoregression on the dataset
from sklearn.linear_model import Lasso

lass_reg = Lasso(alpha=0.001)
lass_reg.fit(x_train, y_train)

lass_pred = lass_reg.predict(x_test)

# calculates the root mean square error for the lasso model
from sklearn.metrics import mean_squared_error

rmse_lass = np.sqrt(mean_squared_error(y_test, lass_pred)).round(2)

print("The root mean square error for the ridge model is {}".format(rmse_lass))

The root mean square error for the ridge model is 0.09


In [107]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(np.array(model.coef_).ravel(), np.array(feat.columns).ravel())
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)

    return weights_df

In [108]:
linear_model_weights = get_weights_df(linear_all, x_train, 'Linear_Model_Weight')

lasso_weights_df = get_weights_df(lass_reg, x_train, 'Lasso_Weight')

ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')

In [109]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')

final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [111]:
print(final_weights)

Features  Linear_Model_Weight  Ridge_Weight  Lasso_Weight
0            T1            -0.003281     -0.018406      0.000000
1          RH_1             0.553547      0.519525      0.017880
2            T2            -0.236178     -0.201397      0.000000
3          RH_2            -0.456698     -0.411071     -0.000000
4            T3             0.290627      0.288087      0.000000
5          RH_3             0.096048      0.095135      0.000000
6            T4             0.028981      0.027384     -0.000000
7          RH_4             0.026386      0.024579      0.000000
8            T5            -0.015657     -0.019853     -0.000000
9          RH_5             0.016006      0.016152      0.000000
10           T6             0.236425      0.217292      0.000000
11         RH_6             0.038049      0.035519     -0.000000
12           T7             0.010319      0.010098     -0.000000
13         RH_7            -0.044614     -0.045977     -0.000000
14           T8             0.10